# IMPORTS

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import QuantileTransformer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import AdaBoostClassifier, StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV


from sklearn.model_selection import cross_validate, cross_val_score

### Data Import

In [ ]:
train_data = "/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv"
df=pd.read_csv(train_data)
df.head(5)

# Looking Into The Data

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
sns.countplot(data=df, x='Made_Purchase');
# data imbalanced

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax = []
for i in range(1, 10):
    ax.append(fig.add_subplot(3, 3, i))

sns.histplot(df['GoogleMetric:Bounce Rates'], bins=20, ax=ax[0], kde=True, linewidth=0)
sns.histplot(df['GoogleMetric:Exit Rates'], bins=20, ax=ax[1], kde=True, linewidth=0)
sns.histplot(df['GoogleMetric:Page Values'], bins=20, ax=ax[2], kde=True, linewidth=0)
    
sns.histplot(data=df, x = 'HomePage', bins=20, ax=ax[3], kde=True, linewidth=0)
sns.histplot(data=df, x = 'HomePage_Duration', bins=20, ax=ax[4], kde=True, linewidth=0)
sns.histplot(data=df, x = 'ProductDescriptionPage', bins=20, ax=ax[5], kde=True, linewidth=0)
    
sns.histplot(data=df, x = 'LandingPage', bins=20, ax=ax[6], kde=True, linewidth=0)
sns.histplot(data=df, x = 'LandingPage_Duration', bins=20, ax=ax[7], kde=True, linewidth=0)
sns.histplot(data=df, x = 'ProductDescriptionPage_Duration', bins=20, ax=ax[8], kde=True, linewidth=0)

plt.show()

# skewed data - need to transform to normal or uniform distribution 

In [ ]:
fig = plt.figure(figsize=(18, 6))
ax = []
for i in range(1, 5):
    ax.append(fig.add_subplot(2, 2, i))

sns.countplot(data=df, x='OS', hue='Made_Purchase', ax=ax[0])
sns.countplot(data=df, x='SearchEngine', hue='Made_Purchase', ax=ax[1])
sns.countplot(data=df, x='Zone', hue='Made_Purchase', ax=ax[2])
sns.countplot(data=df, x='Type of Traffic', hue='Made_Purchase', ax=ax[3])

for i in range(len(ax)):
    ax[i].legend(loc='upper right')
plt.show()

# not much difference based on these things - showing dataimbalnce

In [ ]:
fig = plt.figure(figsize=(18, 15))

ax = []
for i in range(1, 11):
    ax.append(fig.add_subplot(4, 3, i))

sns.countplot(data=df, x='OS', ax=ax[0])
sns.countplot(data=df, x='SearchEngine', ax=ax[1])
sns.countplot(data=df, x='Zone', ax=ax[2])
sns.countplot(data=df, x='Type of Traffic', ax=ax[3])
sns.countplot(data=df, x='CustomerType', ax=ax[4])
sns.countplot(data=df, x='Gender', ax=ax[5])
sns.countplot(data=df, x='Marital Status', ax=ax[6])
sns.countplot(data=df, x='Cookies Setting', ax=ax[7])
sns.countplot(data=df, x='Month_SeasonalPurchase', ax=ax[8])
sns.countplot(data=df, x='Education', ax=ax[9])
plt.show()

# cookie setting, marital status gender and education have all equal number of data points in the data set

# Split The Data

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=41)

for train_index, test_index in sss.split(df, df['Made_Purchase']):
    train, test = df.loc[train_index], df.loc[test_index]

### train test (validation) split

In [ ]:
X = train.drop('Made_Purchase', axis = 1).reset_index().drop('index', axis=1)
y = train['Made_Purchase'].reset_index().drop('index', axis=1)
X_test = test.drop('Made_Purchase', axis = 1).reset_index().drop('index', axis=1)
y_test = test['Made_Purchase'].reset_index().drop('index', axis=1)

### Splitting Variable Set - Categorical / numerical 

In [ ]:
numericVar = ['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
       'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
       'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates',
       'GoogleMetric:Page Values', 'SeasonalPurchase',
       'OS', 'SearchEngine', 'Zone','Type of Traffic', 'WeekendPurchase']
one_hot_cats =['Month_SeasonalPurchase', 'CustomerType', 
               'Gender', 'Cookies Setting',
               'Education', 'Marital Status']

# Preprocessing Function and Pipelines

In [ ]:
def myPreprocessing(df):   
    '''
    putting Nan on improper values
    Input - dataframe - incorrect values
    Output - dataframe - nan replaced on improper values
    '''
    df.loc[(df.HomePage == 0) & (df.HomePage_Duration > 0), 'HomePage'] = np.nan
    df.loc[(df.HomePage > 0) & (df.HomePage_Duration == 0), 'HomePage_Duration'] = np.nan
    df.loc[(df.LandingPage == 0) & (df.LandingPage_Duration > 0), 'LandingPage'] = np.nan
    df.loc[(df.LandingPage > 0) & (df.LandingPage_Duration == 0), 'LandingPage_Duration'] = np.nan
    df.loc[(df.ProductDescriptionPage == 0) & (df.ProductDescriptionPage_Duration > 0), 'ProductDescriptionPage'] = np.nan
    df.loc[(df.ProductDescriptionPage > 0) & (df.ProductDescriptionPage_Duration == 0), 'ProductDescriptionPage_Duration'] = np.nan
    df.loc[(df.SeasonalPurchase > 0) & (df.Month_SeasonalPurchase.isnull()), 'Month_SeasonalPurchase'] = np.nan
    return df

In [ ]:

encoder = ColumnTransformer(
    transformers=[
                  ('ord_encode', 
                   OneHotEncoder(handle_unknown='ignore'), 
                   one_hot_cats),
                 ],remainder='passthrough')

impute = Pipeline(
    steps=[('encoder', encoder),
           ('imputer', IterativeImputer(random_state=0)),
           ('scaler', StandardScaler()),
           ('normaliser', QuantileTransformer(n_quantiles=100, output_distribution='normal'))
          ])

## Fitting the pipelines - converting it back to Dataframe to peek

In [ ]:
X = myPreprocessing(X)
X = impute.fit_transform(X, y)
X_test = impute.transform(X_test)

columns = list(encoder.transformers_[0][1].get_feature_names_out()) + numericVar
X = pd.DataFrame(X, columns = columns)
X_test = pd.DataFrame(X_test, columns = columns)

# Removing Outliers From Training Set
There are abnormal values - like abnormal amount of time in pages duration removing them is essential 

In [ ]:
outlier = IsolationForest(n_estimators=1000, contamination = 'auto', random_state=0)
L = outlier.fit_predict(X)

unique, counts = np.unique(L, return_counts=True)
unique, counts

In [ ]:
X['Outlier'] = L
X['label'] = y

X_New = X.loc[X.Outlier == 1].reset_index().drop(['Outlier', 'index', 'label'], axis=1)
y_New = X.loc[X.Outlier == 1].reset_index()['label']

In [ ]:
X_New.describe().T

# Balancing The Dataset - SMOTE

In [ ]:
rus = SMOTE(random_state=0)

X, y = rus.fit_resample(X_New, y_New)
print(y.value_counts(), y_New.value_counts(), sep = '\n')

In [ ]:
dataCopy = X.copy()
dataCopy['Made_Purchase'] = y.values
plt.figure(figsize = (18, 10))
sns.heatmap(dataCopy.corr(), annot=False,cmap='Greys', vmin=-0.2, vmax=1);

In [ ]:
# clf = SVC(C=10)
# 0.646

# clf1= DecisionTreeClassifier(min_samples_split = 10,min_impurity_decrease=0.01, max_depth=3)
# 0.6732

# estimators = [
#     ('adaboost', AdaBoostClassifier(clf1, n_estimators=100, learning_rate=0.1)),
#    ('rf', RandomForestClassifier(n_estimators=50, random_state=42))
#  ]
# clf = StackingClassifier(estimators=estimators,  passthrough=True, cv=10,
#                        final_estimator = XGBClassifier(n_estimators=100, learning_rate=0.01, random_state=42))
# 0.7461825585341024

estimators = [
    ('adaboost', AdaBoostClassifier(n_estimators=100, learning_rate=0.1)),
    ('rf', RandomForestClassifier(n_estimators=40, random_state=0))
 ]

clf = StackingClassifier(estimators=estimators, passthrough=True,
                         final_estimator = XGBClassifier(n_estimators=150, learning_rate=0.01, random_state=0))
# 0.7523744911804613

clf.fit(X, y)

y_hat = clf.predict(X_test)
score1 = clf.score(X_test, y_test)
print(score1)

Cross validate the classifier on train set

In [ ]:
score = cross_val_score(clf, X, y, scoring='f1_micro', cv=10)
print('Score: ', score.mean() )

In [ ]:
cr = classification_report(y_test, y_hat)
print('Classification Report - Test Set \n\n', cr)

## Hyperparameter Tuning
hyper parameter tuning of this classifier takes more than 20/30 mins. Tuned classifier has been used above as final, and the tuning part is commented on .

In [ ]:
# hyper parameter tuning
n_estimator = [5, 10, 20, 30, 40, 50, 70, 100, 150, 200, 250, 500]
learning_rate = [0.1, 0.01, 0.001, 1, 10]
param = {
    'rf__n_estimators': n_estimator,
    'adaboost__n_estimators':n_estimator,
    'final_estimator__n_estimator': n_estimator,
    'adaboost__learning_rate':learning_rate, 
    'final_estimator__learning_rate':learning_rate,
}

tuned_clf = RandomizedSearchCV(clf,
                            param_distributions= param,
                            cv=10,
                            scoring='f1_micro',
                            n_iter=10,
                            n_jobs=-1,
                            error_score = 'raise',
                            refit=True, 
                            random_state = 42)
# tuned_clf.fit(X, y)
# y_hat_tuned = tuned_clf.predict(X_test)
# score2 = tuned_clf.score(X_test, y_test)
# print(score2)
# print(tuned_clf.best_estimator_)
# tuned = tuned_clf.best_estimator_

# Predicting Finally on Test Data

In [ ]:
test_data = "/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv"
test_df = pd.read_csv(test_data)
test_df = myPreprocessing(test_df)
test_df = pd.DataFrame(impute.transform(test_df), columns = columns)
y_pred = clf.predict(test_df)

output = pd.DataFrame(y_pred.astype(bool))
output = output.reset_index()
output = output.rename(columns = {'index':'id',0:'Made_Purchase'})
output.Made_Purchase.value_counts()

## Submission File

In [ ]:
output.to_csv('submission.csv',index = False)